In [1]:
import pandas as pd
import numpy as np
import sqlite3
import requests
import json
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")
print(f"Today's date: {datetime.now().strftime('%B %d, %Y')}")

All libraries imported successfully!
Today's date: February 22, 2026


In [2]:
# Seed for reproducibility
np.random.seed(42)

# Product categories and brands
categories = ['Laptops', 'Smartphones', 'Tablets', 'Headphones', 'Cameras']
brands = {
    'Laptops': ['Dell', 'HP', 'Lenovo', 'Apple', 'Asus'],
    'Smartphones': ['Samsung', 'Apple', 'OnePlus', 'Xiaomi', 'Google'],
    'Tablets': ['Apple', 'Samsung', 'Lenovo', 'Microsoft', 'Amazon'],
    'Headphones': ['Sony', 'Bose', 'JBL', 'Sennheiser', 'Apple'],
    'Cameras': ['Canon', 'Nikon', 'Sony', 'Fujifilm', 'Panasonic']
}

base_prices = {
    'Laptops': (40000, 150000),
    'Smartphones': (10000, 80000),
    'Tablets': (15000, 70000),
    'Headphones': (1000, 30000),
    'Cameras': (20000, 120000)
}

# Generate 100 products
products = []
for i in range(100):
    category = np.random.choice(categories)
    brand = np.random.choice(brands[category])
    min_price, max_price = base_prices[category]
    base_price = np.random.randint(min_price, max_price)
    
    product = {
        'product_id': f'PRD{i+1:04d}',
        'product_name': f'{brand} {category[:-1]} Model {np.random.randint(100, 999)}',
        'category': category,
        'brand': brand,
        'base_price': base_price,
        'current_price': base_price * np.random.uniform(0.85, 1.15),
        'competitor_price': base_price * np.random.uniform(0.80, 1.20),
        'rating': round(np.random.uniform(3.0, 5.0), 1),
        'reviews_count': np.random.randint(10, 5000),
        'stock_status': np.random.choice(
            ['In Stock', 'Low Stock', 'Out of Stock'],
            p=[0.7, 0.2, 0.1]
        ),
        'discount_percent': np.random.choice(
            [0, 5, 10, 15, 20, 25, 30],
            p=[0.3, 0.1, 0.2, 0.15, 0.1, 0.1, 0.05]
        ),
        'date_collected': datetime.now().strftime('%Y-%m-%d'),
        'time_collected': datetime.now().strftime('%H:%M:%S')
    }
    product['current_price'] = round(product['current_price'], 2)
    product['competitor_price'] = round(product['competitor_price'], 2)
    products.append(product)

df_products = pd.DataFrame(products)
print(f"Generated {len(df_products)} products!")
print(f"\nSample data:")
df_products.head()

Generated 100 products!

Sample data:


,product_id,product_name,category,brand,base_price,current_price,competitor_price,rating,reviews_count,stock_status,discount_percent,date_collected,time_collected
0,PRD0001,Apple Headphone Model 206,Headphones,Apple,6390,6926.17,6637.55,3.9,4436,In Stock,5,2026-02-22,12:05:38
1,PRD0002,Amazon Tablet Model 443,Tablets,Amazon,15769,17341.69,13954.55,3.4,1194,In Stock,10,2026-02-22,12:05:38
2,PRD0003,Sony Headphone Model 158,Headphones,Sony,9666,9375.62,7913.23,4.9,1816,In Stock,20,2026-02-22,12:05:38
3,PRD0004,Microsoft Tablet Model 931,Tablets,Microsoft,16267,16104.80,18609.06,4.4,2898,In Stock,25,2026-02-22,12:05:38
4,PRD0005,Bose Headphone Model 876,Headphones,Bose,11233,9601.85,10023.85,3.5,1488,In Stock,20,2026-02-22,12:05:38


In [3]:
print("DATASET OVERVIEW")
print("=" * 45)
print(f"Total Products    : {len(df_products)}")
print(f"Categories        : {df_products['category'].nunique()}")
print(f"Brands            : {df_products['brand'].nunique()}")
print(f"Price Range       : Rs.{df_products['current_price'].min():,.0f} - Rs.{df_products['current_price'].max():,.0f}")
print(f"Avg Rating        : {df_products['rating'].mean():.1f}")
print(f"\nProducts per Category:")
print(df_products['category'].value_counts())
print(f"\nStock Status:")
print(df_products['stock_status'].value_counts())

DATASET OVERVIEW
Total Products    : 100
Categories        : 5
Brands            : 19
Price Range       : Rs.3,624 - Rs.159,743
Avg Rating        : 4.0

Products per Category:
category
Tablets        26
Headphones     25
Laptops        17
Smartphones    16
Cameras        16
Name: count, dtype: int64

Stock Status:
stock_status
In Stock        74
Low Stock       17
Out of Stock     9
Name: count, dtype: int64


In [4]:
# Round all prices to 2 decimal places
df_products['current_price'] = df_products['current_price'].round(2)
df_products['competitor_price'] = df_products['competitor_price'].round(2)
df_products['base_price'] = df_products['base_price'].round(2)

# Check for any issues
print("DATA QUALITY CHECK")
print("=" * 45)
print(f"Missing values    : {df_products.isnull().sum().sum()}")
print(f"Duplicate rows    : {df_products.duplicated().sum()}")
print(f"Negative prices   : {(df_products['current_price'] < 0).sum()}")
print(f"Invalid ratings   : {((df_products['rating'] < 0) | (df_products['rating'] > 5)).sum()}")
print("\nAll checks passed!" if df_products.isnull().sum().sum() == 0 else "\nIssues found!")

DATA QUALITY CHECK
Missing values    : 0
Duplicate rows    : 0
Negative prices   : 0
Invalid ratings   : 0

All checks passed!


In [5]:
# Price difference vs competitor
df_products['price_vs_competitor'] = (
    df_products['current_price'] - df_products['competitor_price']
).round(2)

# Are we cheaper or more expensive than competitor?
df_products['price_position'] = df_products['price_vs_competitor'].apply(
    lambda x: 'Cheaper' if x < -100 else ('More Expensive' if x > 100 else 'Competitive')
)

# Discount amount in rupees
df_products['discount_amount'] = (
    df_products['base_price'] * df_products['discount_percent'] / 100
).round(2)

# Price after discount
df_products['final_price'] = (
    df_products['current_price'] - df_products['discount_amount']
).round(2)

# Competitiveness score (0-100)
df_products['competitiveness_score'] = (
    (df_products['rating'] * 15) +
    (df_products['reviews_count'] / df_products['reviews_count'].max() * 30) +
    ((df_products['competitor_price'] - df_products['current_price']) /
     df_products['competitor_price'] * 55)
).round(1).clip(0, 100)

# Flag products with high competitor advantage
df_products['alert'] = df_products.apply(
    lambda row: 'COMPETITOR CHEAPER' if row['price_vs_competitor'] > 2000
    else ('LOW STOCK' if row['stock_status'] == 'Low Stock'
    else ('OUT OF STOCK' if row['stock_status'] == 'Out of Stock'
    else 'OK')), axis=1
)

print("Business metrics added successfully!")
print(f"\nPrice Position Summary:")
print(df_products['price_position'].value_counts())
print(f"\nAlert Summary:")
print(df_products['alert'].value_counts())

Business metrics added successfully!

Price Position Summary:
price_position
Cheaper           54
More Expensive    44
Competitive        2
Name: count, dtype: int64

Alert Summary:
alert
OK                    51
COMPETITOR CHEAPER    34
LOW STOCK             10
OUT OF STOCK           5
Name: count, dtype: int64


In [6]:
# Generate 7 days of price history for each product
price_history = []

for _, product in df_products.iterrows():
    for days_ago in range(7, 0, -1):
        date = (datetime.now() - timedelta(days=days_ago)).strftime('%Y-%m-%d')
        
        # Simulate realistic price fluctuation
        price_variation = np.random.uniform(-0.08, 0.08)
        historical_price = product['base_price'] * (1 + price_variation)
        
        price_history.append({
            'product_id': product['product_id'],
            'product_name': product['product_name'],
            'category': product['category'],
            'brand': product['brand'],
            'date': date,
            'price': round(historical_price, 2),
            'competitor_price': round(
                product['base_price'] * np.random.uniform(0.80, 1.20), 2)
        })

df_history = pd.DataFrame(price_history)
print(f"Price history generated!")
print(f"Total records: {len(df_history)}")
print(f"Date range: {df_history['date'].min()} to {df_history['date'].max()}")
df_history.head(10)

Price history generated!
Total records: 700
Date range: 2026-02-15 to 2026-02-21


,product_id,product_name,category,brand,date,price,competitor_price
0,PRD0001,Apple Headphone Model 206,Headphones,Apple,2026-02-15,5984.09,5966.63
1,PRD0001,Apple Headphone Model 206,Headphones,Apple,2026-02-16,5955.99,7037.30
2,PRD0001,Apple Headphone Model 206,Headphones,Apple,2026-02-17,6157.20,7405.83
3,PRD0001,Apple Headphone Model 206,Headphones,Apple,2026-02-18,6417.17,7158.72
4,PRD0001,Apple Headphone Model 206,Headphones,Apple,2026-02-19,6879.66,7258.50
5,PRD0001,Apple Headphone Model 206,Headphones,Apple,2026-02-20,6765.21,6154.81
6,PRD0001,Apple Headphone Model 206,Headphones,Apple,2026-02-21,6442.88,5760.94
7,PRD0002,Amazon Tablet Model 443,Tablets,Amazon,2026-02-15,15002.28,15803.74
8,PRD0002,Amazon Tablet Model 443,Tablets,Amazon,2026-02-16,16008.81,14755.19
9,PRD0002,Amazon Tablet Model 443,Tablets,Amazon,2026-02-17,15944.21,18212.95


In [7]:
# Market analysis by category
category_analysis = df_products.groupby('category').agg(
    Total_Products=('product_id', 'count'),
    Avg_Price=('current_price', 'mean'),
    Min_Price=('current_price', 'min'),
    Max_Price=('current_price', 'max'),
    Avg_Rating=('rating', 'mean'),
    Total_Reviews=('reviews_count', 'sum'),
    Avg_Discount=('discount_percent', 'mean'),
    Out_of_Stock=('stock_status', lambda x: (x == 'Out of Stock').sum())
).reset_index()

category_analysis['Avg_Price'] = category_analysis['Avg_Price'].round(0)
category_analysis['Avg_Rating'] = category_analysis['Avg_Rating'].round(1)
category_analysis['Avg_Discount'] = category_analysis['Avg_Discount'].round(1)

print("CATEGORY MARKET ANALYSIS")
print("=" * 65)
print(category_analysis.to_string(index=False))

CATEGORY MARKET ANALYSIS
   category  Total_Products  Avg_Price  Min_Price  Max_Price  Avg_Rating  Total_Reviews  Avg_Discount  Out_of_Stock
    Cameras              16    69021.0   20741.28  108071.93         3.7          40533          13.8             0
 Headphones              25    17433.0    3623.58   30283.57         4.1          71409           9.6             3
    Laptops              17    98192.0   46436.35  159743.01         4.1          40354           8.2             1
Smartphones              16    48171.0   12764.89   84663.17         4.0          56537          10.0             3
    Tablets              26    36581.0   13430.96   72523.31         3.9          62692          14.0             2


In [8]:
# Brand performance across all categories
brand_analysis = df_products.groupby('brand').agg(
    Total_Products=('product_id', 'count'),
    Avg_Price=('current_price', 'mean'),
    Avg_Rating=('rating', 'mean'),
    Total_Reviews=('reviews_count', 'sum'),
    Avg_Competitiveness=('competitiveness_score', 'mean'),
    Cheaper_Than_Competitor=('price_position',
                              lambda x: (x == 'Cheaper').sum())
).reset_index().sort_values('Avg_Rating', ascending=False)

brand_analysis['Avg_Price'] = brand_analysis['Avg_Price'].round(0)
brand_analysis['Avg_Rating'] = brand_analysis['Avg_Rating'].round(1)
brand_analysis['Avg_Competitiveness'] = brand_analysis['Avg_Competitiveness'].round(1)

print("BRAND PERFORMANCE ANALYSIS")
print("=" * 65)
print(brand_analysis.head(10).to_string(index=False))

BRAND PERFORMANCE ANALYSIS
     brand  Total_Products  Avg_Price  Avg_Rating  Total_Reviews  Avg_Competitiveness  Cheaper_Than_Competitor
        HP               4    87148.0         4.3          10045                 84.0                        3
    Lenovo               9    58271.0         4.3          22123                 81.6                        7
    Xiaomi               3    75891.0         4.2          12490                 80.5                        0
   Samsung               5    55984.0         4.2          18293                 86.8                        5
       JBL               2    24305.0         4.2           3811                 69.1                        0
      Sony              12    31435.0         4.2          28899                 77.1                        8
Sennheiser               4    18877.0         4.2          14778                 88.2                        3
      Asus               4   104103.0         4.1           6822                 72.0

In [9]:
# Products where competitor is significantly cheaper
competitor_analysis = df_products[
    df_products['price_vs_competitor'] > 0
].sort_values('price_vs_competitor', ascending=False)

print("COMPETITOR PRICE ADVANTAGE ANALYSIS")
print("=" * 65)
print(f"Products where competitor is cheaper: {len(competitor_analysis)}")
print(f"Average price gap: Rs.{competitor_analysis['price_vs_competitor'].mean():,.0f}")
print(f"Maximum price gap: Rs.{competitor_analysis['price_vs_competitor'].max():,.0f}")
print(f"\nTop 10 Products Where Competitor Has Biggest Advantage:")
print(competitor_analysis[['product_name', 'category', 'current_price',
                            'competitor_price', 'price_vs_competitor']].head(10).to_string(index=False))

COMPETITOR PRICE ADVANTAGE ANALYSIS
Products where competitor is cheaper: 46
Average price gap: Rs.5,443
Maximum price gap: Rs.22,511

Top 10 Products Where Competitor Has Biggest Advantage:
                product_name    category  current_price  competitor_price  price_vs_competitor
      Canon Camera Model 510     Cameras       92997.82          70487.11             22510.71
 Xiaomi Smartphone Model 315 Smartphones       82798.65          64065.40             18733.25
       Dell Laptop Model 982     Laptops      114593.70          99102.24             15491.46
  Panasonic Camera Model 978     Cameras      103341.81          88675.20             14666.61
       Dell Laptop Model 823     Laptops      122413.67         107872.30             14541.37
      Nikon Camera Model 441     Cameras      108071.93          95274.75             12797.18
      Apple Tablet Model 776     Tablets       72523.31          62240.90             10282.41
OnePlus Smartphone Model 840 Smartphones       30

In [10]:
# Create database
conn = sqlite3.connect('market_intelligence.db')
cursor = conn.cursor()

# Create products table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        product_id TEXT PRIMARY KEY,
        product_name TEXT,
        category TEXT,
        brand TEXT,
        base_price REAL,
        current_price REAL,
        competitor_price REAL,
        rating REAL,
        reviews_count INTEGER,
        stock_status TEXT,
        discount_percent REAL,
        discount_amount REAL,
        final_price REAL,
        price_vs_competitor REAL,
        price_position TEXT,
        competitiveness_score REAL,
        alert TEXT,
        date_collected TEXT,
        time_collected TEXT
    )
''')

# Create price history table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS price_history (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        product_id TEXT,
        product_name TEXT,
        category TEXT,
        brand TEXT,
        date TEXT,
        price REAL,
        competitor_price REAL
    )
''')

# Create market summary table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS market_summary (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT,
        category TEXT,
        avg_price REAL,
        avg_rating REAL,
        total_products INTEGER,
        out_of_stock_count INTEGER
    )
''')

conn.commit()
print("Database and tables created successfully!")

Database and tables created successfully!


In [11]:
# Load products into database
df_products.to_sql('products', conn, if_exists='replace', index=False)
print(f"Products loaded: {len(df_products)} records")

# Load price history
df_history.to_sql('price_history', conn, if_exists='replace', index=False)
print(f"Price history loaded: {len(df_history)} records")

# Create and load market summary
market_summary = df_products.groupby('category').agg(
    avg_price=('current_price', 'mean'),
    avg_rating=('rating', 'mean'),
    total_products=('product_id', 'count'),
    out_of_stock_count=('stock_status',
                         lambda x: (x == 'Out of Stock').sum())
).reset_index()
market_summary['date'] = datetime.now().strftime('%Y-%m-%d')
market_summary.to_sql('market_summary', conn,
                       if_exists='replace', index=False)
print(f"Market summary loaded: {len(market_summary)} records")

conn.commit()
print("\nAll data loaded into database successfully!")

Products loaded: 100 records
Price history loaded: 700 records
Market summary loaded: 5 records

All data loaded into database successfully!


In [12]:
def run_sql(query, title):
    print(f"\n{'='*55}")
    print(f"{title}")
    print('='*55)
    result = pd.read_sql_query(query, conn)
    print(result.to_string(index=False))
    return result

# Query 1 - Overview
run_sql("""
    SELECT
        category,
        COUNT(*) as Products,
        ROUND(AVG(current_price), 0) as Avg_Price,
        ROUND(AVG(rating), 1) as Avg_Rating,
        SUM(CASE WHEN alert != 'OK' THEN 1 ELSE 0 END) as Alerts
    FROM products
    GROUP BY category
    ORDER BY Avg_Price DESC
""", "MARKET OVERVIEW BY CATEGORY")

# Query 2 - Alerts
run_sql("""
    SELECT
        product_name,
        category,
        current_price,
        competitor_price,
        price_vs_competitor,
        alert
    FROM products
    WHERE alert != 'OK'
    ORDER BY price_vs_competitor DESC
    LIMIT 10
""", "TOP 10 PRODUCTS NEEDING ATTENTION")

# Query 3 - Best rated products
run_sql("""
    SELECT
        product_name,
        brand,
        category,
        current_price,
        rating,
        reviews_count
    FROM products
    ORDER BY rating DESC, reviews_count DESC
    LIMIT 10
""", "TOP 10 HIGHEST RATED PRODUCTS")

conn.close()
print("\nAll SQL queries complete!")


MARKET OVERVIEW BY CATEGORY
   category  Products  Avg_Price  Avg_Rating  Alerts
    Laptops        17    98192.0         4.1       8
    Cameras        16    69021.0         3.7       9
Smartphones        16    48171.0         4.0      11
    Tablets        26    36581.0         3.9      13
 Headphones        25    17433.0         4.1       8

TOP 10 PRODUCTS NEEDING ATTENTION
                product_name    category  current_price  competitor_price  price_vs_competitor              alert
      Canon Camera Model 510     Cameras       92997.82          70487.11             22510.71 COMPETITOR CHEAPER
 Xiaomi Smartphone Model 315 Smartphones       82798.65          64065.40             18733.25 COMPETITOR CHEAPER
       Dell Laptop Model 982     Laptops      114593.70          99102.24             15491.46 COMPETITOR CHEAPER
  Panasonic Camera Model 978     Cameras      103341.81          88675.20             14666.61 COMPETITOR CHEAPER
       Dell Laptop Model 823     Laptops      12

In [13]:
# Save all datasets
df_products.to_csv('products_data.csv', index=False)
df_products.to_excel('products_data.xlsx', index=False)
df_history.to_csv('price_history.csv', index=False)
category_analysis.to_csv('category_analysis.csv', index=False)
brand_analysis.to_csv('brand_analysis.csv', index=False)

print("All files saved!")
print("\nFiles created:")
files = ['products_data.csv', 'products_data.xlsx',
         'price_history.csv', 'category_analysis.csv',
         'brand_analysis.csv', 'market_intelligence.db']
for f in files:
    size = os.path.getsize(f) / 1024
    print(f"  {f} ({size:.1f} KB)")

All files saved!

Files created:
  products_data.csv (16.3 KB)
  products_data.xlsx (16.2 KB)
  price_history.csv (53.7 KB)
  category_analysis.csv (0.4 KB)
  brand_analysis.csv (0.7 KB)
  market_intelligence.db (100.0 KB)


In [14]:
def refresh_market_data():
    """
    Call this function anytime to get fresh market data.
    In production this would connect to real APIs or scrapers.
    """
    print(f"Refreshing market data at {datetime.now().strftime('%H:%M:%S')}...")

    conn = sqlite3.connect('market_intelligence.db')

    # Simulate price updates
    updated_products = df_products.copy()
    updated_products['current_price'] = updated_products['current_price'].apply(
        lambda x: round(x * np.random.uniform(0.97, 1.03), 2))
    updated_products['competitor_price'] = updated_products['competitor_price'].apply(
        lambda x: round(x * np.random.uniform(0.97, 1.03), 2))
    updated_products['date_collected'] = datetime.now().strftime('%Y-%m-%d')
    updated_products['time_collected'] = datetime.now().strftime('%H:%M:%S')

    # Update database
    updated_products.to_sql('products', conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()

    print(f"Data refreshed successfully!")
    print(f"Updated {len(updated_products)} product prices")
    return updated_products

# Test the refresh function
refresh_market_data()

Refreshing market data at 12:08:54...
Data refreshed successfully!
Updated 100 product prices


,product_id,product_name,category,brand,base_price,current_price,competitor_price,rating,reviews_count,stock_status,discount_percent,date_collected,time_collected,price_vs_competitor,price_position,discount_amount,final_price,competitiveness_score,alert
0,PRD0001,Apple Headphone Model 206,Headphones,Apple,6390,7046.19,6760.22,3.9,4436,In Stock,5,2026-02-22,12:08:54,288.62,More Expensive,319.50,6606.67,82.8,OK
1,PRD0002,Amazon Tablet Model 443,Tablets,Amazon,15769,17486.95,13716.81,3.4,1194,In Stock,10,2026-02-22,12:08:54,3387.14,More Expensive,1576.90,15764.79,44.8,COMPETITOR CHEAPER
2,PRD0003,Sony Headphone Model 158,Headphones,Sony,9666,9474.87,7897.38,4.9,1816,In Stock,20,2026-02-22,12:08:54,1462.39,More Expensive,1933.20,7442.42,74.3,OK
3,PRD0004,Microsoft Tablet Model 931,Tablets,Microsoft,16267,15748.91,18962.70,4.4,2898,In Stock,25,2026-02-22,12:08:54,-2504.26,Cheaper,4066.75,12038.05,90.8,OK
4,PRD0005,Bose Headphone Model 876,Headphones,Bose,11233,9448.71,9763.66,3.5,1488,In Stock,20,2026-02-22,12:08:54,-422.00,Cheaper,2246.60,7355.25,63.8,OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,PRD0096,Apple Smartphone Model 517,Smartphones,Apple,18666,21050.22,19084.86,3.1,4868,In Stock,0,2026-02-22,12:08:54,1693.17,More Expensive,0.00,21009.26,71.0,OK
96,PRD0097,Apple Headphone Model 260,Headphones,Apple,23359,22325.97,23860.00,4.3,2213,In Stock,10,2026-02-22,12:08:54,-1717.29,Cheaper,2335.90,20058.80,81.7,OK
97,PRD0098,Apple Laptop Model 788,Laptops,Apple,59816,59327.09,69645.92,4.1,4432,In Stock,0,2026-02-22,12:08:54,-7600.30,Cheaper,0.00,60844.33,94.3,OK
98,PRD0099,Lenovo Tablet Model 822,Tablets,Lenovo,48719,46236.82,47988.64,4.2,1510,In Stock,10,2026-02-22,12:08:54,-1582.22,Cheaper,4871.90,41645.79,73.9,OK


In [15]:
import os

print("=" * 55)
print("DAY 7 COMPLETE - FILE CHECK")
print("=" * 55)

files_to_check = [
    'Day7_Data_Collection.ipynb',
    'products_data.csv',
    'products_data.xlsx',
    'price_history.csv',
    'category_analysis.csv',
    'brand_analysis.csv',
    'market_intelligence.db',
]

for f in files_to_check:
    status = "FOUND" if os.path.exists(f) else "MISSING"
    size = f"({os.path.getsize(f)/1024:.1f} KB)" if os.path.exists(f) else ""
    print(f"  {status}  {f} {size}")

print("\nWHAT YOU BUILT TODAY:")
print("  100 products across 5 categories")
print("  7 days of price history per product")
print("  Business metrics and competitor analysis")
print("  SQLite database with 3 tables")
print("  Data refresh automation function")
print("\nDAY 7 - 100% COMPLETE!")
print("=" * 55)

DAY 7 COMPLETE - FILE CHECK
  FOUND  Day7_Data_Collection.ipynb (39.8 KB)
  FOUND  products_data.csv (16.3 KB)
  FOUND  products_data.xlsx (16.2 KB)
  FOUND  price_history.csv (53.7 KB)
  FOUND  category_analysis.csv (0.4 KB)
  FOUND  brand_analysis.csv (0.7 KB)
  FOUND  market_intelligence.db (100.0 KB)

WHAT YOU BUILT TODAY:
  100 products across 5 categories
  7 days of price history per product
  Business metrics and competitor analysis
  SQLite database with 3 tables
  Data refresh automation function

DAY 7 - 100% COMPLETE!


## What We Built Today
1. Generated realistic market intelligence data for 100 products
2. Added business metrics including competitiveness score and alerts
3. Built 7 days of price history tracking per product
4. Created SQLite database with 3 tables
5. Wrote SQL queries to extract key business insights
6. Built data refresh automation function

## Data Structure
- products table: 100 products with full metrics
- price_history table: 700 records of historical prices
- market_summary table: Category level daily summary

## Key Findings from the Data
- Products where competitor is significantly cheaper need immediate attention
- Some categories have higher average ratings than others
- Stock alerts highlight products needing inventory attention
